In [1]:
pinnacle_path = r"D:\WORKSPACE\dotabet\data\odds\pinnacle.csv"
odds_path = r"D:\WORKSPACE\dotabet\data\odds\odds.csv"

In [2]:
import dotabet
import requests
import csv
import pandas as pd
from datetime import datetime

def format_timestamp(ts):
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

def determine_winner(row):
    if row['radiant_win']:
        return row['radiant_name']
    else:
        return row['dire_name']

def get_team_id(row, team_name_key):
    if row[team_name_key] == row['radiant_name']:
        return row['radiant_team_id']
    elif row[team_name_key] == row['dire_name']:
        return row['dire_team_id']
    return None

In [3]:
df = pd.read_csv(pinnacle_path)
df['Team_1'] = df['Team_1'].apply(dotabet.utils.format_team_name)
df['Team_2'] = df['Team_2'].apply(dotabet.utils.format_team_name)
df['Teams'] = df.apply(lambda x: frozenset([x['Team_1'], x['Team_2']]), axis=1)
df.set_index(['Match_Date', 'Teams'], inplace=True)

In [4]:
endpoint_matches = "https://api.opendota.com/api/proMatches"
completed_dfs = []
for _ in range(5):
    fetched_matches = dotabet.fetch.fetch_data(endpoint_matches)

    api_df = pd.DataFrame(fetched_matches)
    api_df['Match_Date'] = api_df['start_time'].apply(format_timestamp)
    api_df['radiant_name'] = api_df['radiant_name'].apply(dotabet.utils.format_team_name)
    api_df['dire_name'] = api_df['dire_name'].apply(dotabet.utils.format_team_name)
    api_df['Teams'] = api_df.apply(lambda x: frozenset([x['radiant_name'], x['dire_name']]), axis=1)
    api_df = api_df.drop_duplicates(subset=['Match_Date', 'Teams'], keep='first')
    api_df.set_index(['Match_Date', 'Teams'], inplace=True)

    merged_df = df.merge(api_df, how='inner', left_index=True, right_index=True)

    df = df.drop(merged_df.index)

    completed_dfs.append(merged_df)
    endpoint_matches = f"https://api.opendota.com/api/proMatches?less_than_match_id={fetched_matches[-1]['match_id']}"

final = pd.concat(completed_dfs)
final.reset_index(inplace=True)
final['team1_team_id'] = final.apply(lambda row: get_team_id(row, 'Team_1'), axis=1)
final['team2_team_id'] = final.apply(lambda row: get_team_id(row, 'Team_2'), axis=1)
final['winner'] = final.apply(determine_winner, axis=1)
final = final.drop(['Teams', 'League_Name', 'duration', 'start_time', 'radiant_team_id', 'radiant_name', 'version',\
                   'dire_team_id', 'dire_name', 'series_id', 'series_type', 'radiant_score', 'dire_score', 'radiant_win'],axis=1)
odds = pd.read_csv(odds_path)
output = pd.concat([final, odds])
output.to_csv(odds_path, index=False)


df.reset_index(inplace=True)
df = df.drop('Teams',axis=1)
df.to_csv(pinnacle_path, index=False)

In [30]:
nik = ['a', 'b'] + ['c']
nik

['a', 'b', 'c']

In [84]:
import json
df = final

def elo_probability(rating1, rating2):
    if rating1 and rating1:
        return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

with open(r"D:\WORKSPACE\dotabet\data\top_teams\PGL_Wallachia_2024_Season_16669\api_snapshots\api_leagues_16669_teams_group.json", 'r') as f:
    team_info = json.load(f)

team_ratings = {team['team_id']: team['rating'] for team in team_info}

df['team1_rating'] = df['team1_team_id'].map(team_ratings)
df['team2_rating'] = df['team2_team_id'].map(team_ratings)

df['team1_proba'] = df.apply(lambda row: elo_probability(row['team1_rating'], row['team2_rating']), axis=1)
df['team2_proba'] = df.apply(lambda row: elo_probability(row['team2_rating'], row['team1_rating']), axis=1)

def compute_coefficient(probability):
    if probability is None or probability == 0:
        return None  # Or a large value like float('inf') if you prefer
    else:
        return 1 / probability

df['team1_coef'] = df['team1_proba'].apply(compute_coefficient)
df['team2_coef'] = df['team2_proba'].apply(compute_coefficient)

In [85]:
df.to_csv(r"D:\WORKSPACE\dotabet\data\odds\nikita.csv")